In [1]:
import pandas as pd

In [19]:
_data = pd.read_csv('../resources/data.csv', index_col=0, skiprows=[2])

In [21]:
ans_key = _data.head(1)

In [32]:
data = _data.drop(_data.index[0])

,Term,Test,Instructor,LetterGrade,PercentGrade,Q1,Q2,Q3,Q4,Q5,...,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25,Q26,Q27
Participant Code,,,,,,,,,,,,,,,,,,,,,
KEY,NaN,NaN,NaN,NaN,NaN,E,D,D,E,A,...,B,C,B,D,C,C,D,B,A,D
